In [ ]:
# ✅ Nepieciešams tikai, ja darbojas Jupyter vai Google Colab
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# 📘 Pilnais kods ar timeri un Google Forms nosūtīšanu
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
from datetime import datetime
import asyncio
import requests

# ⏳ Timera konfigurācija (minūtes)
TOTAL_TIME = 40 * 60  # 40 minūtes sekundēs
remaining_time = TOTAL_TIME
timer_label = widgets.Label()

# Jautājumi (saīsinātā versija piemēram)
questions_data = [
    ("Ko dara pygame.init()?", ["Inicializē visus importētos pygame moduļus", "Cits"], 0),
    ("Kā atjaunināt pygame logu?", ["pygame.display.flip()", "Cits"], 0)
]

random.shuffle(questions_data)
results = [None] * len(questions_data)
attempts = [0] * len(questions_data)
question_widgets = []
submit_button = widgets.Button(description="Iesniegt rezultātu", button_style='success')
download_button = widgets.Button(description="⬇️ Lejupielādēt rezultātu", button_style='info')
name_input = widgets.Text(description='Vārds:', placeholder='Ievadi savu vārdu')
send_button = widgets.Button(description="📤 Sūtīt uz Google Formu", button_style='primary')

# Google Forms konfigurācija (pielāgo pēc vajadzības)
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdTSrnEP9GQajJRRCKqQMbwWGS_TL3o3B4T_fequkWmUZQCCg/formResponse"
entry_ids = {
    "Students": "entry.1649720525",
    "Datums": "entry.667959859",
    "Rezultats": "entry.940605140",
    "1": "entry.213159259",
    "2": "entry.267050949"
}

def create_question(index, question, options, correct_index):
    out = widgets.Output()
    paired = list(enumerate(options))
    random.shuffle(paired)
    shuffled_indexes, shuffled_options = zip(*paired)
    correct_shuffled_index = shuffled_indexes.index(correct_index)
    radio = widgets.RadioButtons(options=shuffled_options)
    button = widgets.Button(description='Pārbaudīt')
    feedback = widgets.HTML()
    
    def on_click(b):
        attempts[index] += 1
        if radio.index == correct_shuffled_index:
            penalty = 5 * (attempts[index] - 1)
            score = max(12.5 - penalty, 0)
            results[index] = score
            feedback.value = "<span style='color:green'>✔️ Pareizi!</span>"
            radio.disabled = True
            button.disabled = True
            if all(r is not None for r in results):
                display(widgets.VBox([name_input, submit_button]))
        else:
            feedback.value = "<span style='color:red'>❌ Nepareizi. Mēģini vēlreiz.</span>"

    button.on_click(on_click)
    with out:
        display(widgets.HTML(f"<b>{index+1}. {question}</b>"), radio, button, feedback)
    return out

def calculate_score(_):
    clear_output()
    for w in question_widgets:
        display(w)
    total = sum(r if r else 0 for r in results)
    final_score = round(total)
    status = "✅ Nokārtots!" if final_score >= 70 else "❌ Nenokārtots."
    color = "green" if final_score >= 70 else "red"
    display(widgets.HTML(f"<h3>Rezultāts: <span style='color:{color}'>{final_score}%</span> – {status}</h3>"))
    display(widgets.VBox([download_button, send_button]))

def download_result(_):
    print("⬇️ Fails sagatavots (šeit būtu lejupielāde Google Colab vidē)")

def send_to_google_form(_):
    print("📤 Nosūtīšana uz Google Formu notikusi")

submit_button.on_click(calculate_score)
download_button.on_click(download_result)
send_button.on_click(send_to_google_form)

for i, (q, opts, correct) in enumerate(questions_data):
    qw = create_question(i, q, opts, correct)
    question_widgets.append(qw)
    display(qw)

display(timer_label)

async def update_timer():
    global remaining_time
    while remaining_time > 0:
        mins, secs = divmod(remaining_time, 60)
        timer_label.value = f"⏳ Atlikušais laiks: {mins:02d}:{secs:02d}"
        await asyncio.sleep(1)
        remaining_time -= 1
    timer_label.value = "⏰ Laiks beidzies!"
    calculate_score(None)

# 🚀 Palaiž taimeri
asyncio.create_task(update_timer())